In [14]:
import asyncio
from typing import Dict, Any, Optional

from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.genai import types
from google.adk.events import Event

# --- Define Tool Functions ---
# These functions simulate the actions of the specialist agents.

def booking_handler(request: str) -> str:
    """
    Handles booking requests for flights and hotels.
    Args:
        request: The user's request for a booking.
    Returns:
        A confirmation message that the booking was handled.
    """
    print("-------------------------- Booking Handler Called ----------------------------")
    return f"Booking action for '{request}' has been simulated."

def info_handler(request: str) -> str:
    """
    Handles general information requests.
    Args:
        request: The user's question.
    Returns:
        A message indicating the information request was handled.
    """
    print("-------------------------- Info Handler Called ----------------------------")
    return f"Information request for '{request}'. Result: Simulated information retrieval."

# --- Create Tools from Functions ---
booking_tool = FunctionTool(booking_handler)
info_tool = FunctionTool(info_handler)

# Define specialized sub-agents equipped with their respective tools
booking_agent = Agent(
    name="Booker",
    model="gemini-2.0-flash",
    description="A specialized agent that handles all flight and hotel booking requests by calling the booking tool.",
    tools=[booking_tool]
)

info_agent = Agent(
    name="Info",
    model="gemini-2.0-flash",
    description="A specialized agent that provides general information and answers user questions by calling the info tool.",
    tools=[info_tool]
)

# Define the parent agent with explicit delegation instructions
coordinator = Agent(
    name="Coordinator",
    model="gemini-2.0-flash",
    instruction=(
        "You are the main coordinator. Your only task is to analyze incoming user requests "
        "and delegate them to the appropriate specialist agent. Do not try to answer the user directly.\n"
        "- For any requests related to booking flights or hotels, delegate to the 'Booker' agent.\n"
        "- For all other general information questions, delegate to the 'Info' agent."
    ),
    description="A coordinator that routes user requests to the correct specialist agent.",
    # The presence of sub_agents enables LLM-driven delegation (Auto-Flow) by default.
    sub_agents=[booking_agent, info_agent]
)

# --- Execution Logic ---

print("--- Google ADK Routing Example (ADK Auto-Flow Style) ---")
print("Note: This requires Google ADK installed and authenticated.")

# Create runner with app_name
APP_NAME = "coordinator_app"
USER_ID = "user_123"
SESSION_ID = "session_001"

runner = InMemoryRunner(agent=coordinator, app_name=APP_NAME)

# Create session manually first
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: {session.id}")

async def run_coordinator(request: str):
    """Runs the coordinator agent with a given request and delegates."""
    print(f"\n--- Running Coordinator with request: '{request}' ---")
    final_result = ""
    try:
        # Use run_async with the pre-created session
        async for event in runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=types.Content(
                role='user',
                parts=[types.Part(text=request)]
            ),
        ):
            # Print all events for debugging
            print(f"Event: {event.type if hasattr(event, 'type') else 'unknown'}")
            
            if event.is_final_response() and event.content:
                if hasattr(event.content, 'text') and event.content.text:
                    final_result = event.content.text
                elif event.content.parts:
                    text_parts = [part.text for part in event.content.parts if part.text]
                    final_result = "".join(text_parts)

        print(f"Coordinator Final Response: {final_result}")
        return final_result
    except Exception as e:
        print(f"An error occurred while processing your request: {e}")
        import traceback
        traceback.print_exc()
        return f"An error occurred while processing your request: {e}"



--- Google ADK Routing Example (ADK Auto-Flow Style) ---
Note: This requires Google ADK installed and authenticated.
Session created: session_001


In [15]:
# Example Usage
result_a = await run_coordinator("Book me a hotel in Paris.")
print(f"Final Output A: {result_a}")


--- Running Coordinator with request: 'Book me a hotel in Paris.' ---
Event: unknown
Event: unknown
Event: unknown
-------------------------- Booking Handler Called ----------------------------
Event: unknown
Event: unknown
Coordinator Final Response: OK. I have simulated booking a hotel in Paris.

Final Output A: OK. I have simulated booking a hotel in Paris.



In [16]:
result_b = await run_coordinator("What is the highest mountain in the world?")
print(f"Final Output B: {result_b}")


--- Running Coordinator with request: 'What is the highest mountain in the world?' ---
Event: unknown
Event: unknown
Event: unknown
-------------------------- Info Handler Called ----------------------------
Event: unknown
Event: unknown
Coordinator Final Response: The highest mountain in the world is Mount Everest.

Final Output B: The highest mountain in the world is Mount Everest.



In [17]:
result_c = await run_coordinator("Tell me a random fact.")
print(f"Final Output C: {result_c}")


--- Running Coordinator with request: 'Tell me a random fact.' ---
Event: unknown
-------------------------- Info Handler Called ----------------------------
Event: unknown
Event: unknown
Coordinator Final Response: A group of owls is called a parliament.

Final Output C: A group of owls is called a parliament.



In [18]:
result_d = await run_coordinator("Find flights to Tokyo next month.")
print(f"Final Output D: {result_d}")


--- Running Coordinator with request: 'Find flights to Tokyo next month.' ---
Event: unknown
Event: unknown
Event: unknown
-------------------------- Booking Handler Called ----------------------------
Event: unknown
Event: unknown
Coordinator Final Response: OK. I have simulated finding flights to Tokyo next month.

Final Output D: OK. I have simulated finding flights to Tokyo next month.

